In [ ]:
import os
from collections import defaultdict
import torch
import numpy as np
import scipy.stats
from torch.distributions import constraints
from matplotlib import pyplot
%matplotlib inline

import pyro
import pyro.distributions as dist
from pyro import poutine
from pyro.infer.autoguide import AutoDelta
from pyro.optim import Adam
from pyro.infer import SVI, TraceEnum_ELBO, config_enumerate, infer_discrete

smoke_test = ('CI' in os.environ)
# assert pyro.__version__.startswith('1.6.0')

In [ ]:
data = torch.rand((4,3))

In [ ]:
K = 2  # Fixed number of components.
B = data.shape[1] # fixed number of bags
NP = data.shape[0]
NU = NP
@config_enumerate
def model(dataPos,dataU):
    with pyro.plate('clusters',K):
        locs = pyro.sample('locs', dist.Normal(0., 10.))
        scale = pyro.sample('scale', dist.LogNormal(0., 2.))
    with pyro.plate('bags', B):
        pi = pyro.sample('pi', dist.Dirichlet(torch.ones(K)))
        rho = pyro.sample('rho', dist.Dirichlet(torch.ones(K)))
        alpha = pyro.sample('alpha',dist.Uniform(0,1))
        with pyro.plate('pos_data', NP):
            c = pyro.sample('c', dist.Categorical(pi))
            x = pyro.sample('x', dist.Normal(locs[c], scale[c]), obs=dataU)
        with pyro.plate('unlabeled_data',NU):
            y = pyro.sample('y', dist.Bernoulli(alpha,),)
            p = torch.stack((pi,)*NU,dim=0) * torch.stack((y,)*K,2) + torch.stack((rho,)*NU,dim=0)*(1-torch.stack((y,)*K,2))
            c = pyro.sample('c',dist.Categorical(p))
            x = pyro.sample('x',dist.Normal(locs[c],scale[c]),obs=dataU)

In [ ]:
model(data,data)

In [ ]:
optim = pyro.optim.Adam({'lr': 0.1, 'betas': [0.8, 0.99]})
elbo = TraceEnum_ELBO(max_plate_nesting=1)

In [ ]:
def init_loc_fn(site):
    if site["name"] == "weights":
        # Initialize weights to uniform.
        return torch.ones(K) / K
    if site["name"] == "scale":
        s = torch.ones(K) * (data.var() / 2).sqrt()
        return s
    if site["name"] == "locs":
        m = data[torch.multinomial(torch.ones(len(data)) / len(data), K)]
        return m
    raise ValueError(site["name"])

def initialize(seed):
    global global_guide, svi
    pyro.set_rng_seed(seed)
    pyro.clear_param_store()
    global_guide = AutoDelta(poutine.block(model, expose=['weights', 'locs', 'scale']),
                             init_loc_fn=init_loc_fn)
    svi = SVI(model, global_guide, optim, loss=elbo)
    return svi.loss(model, global_guide, data)

# Choose the best among 100 random initializations.
loss, seed = min((initialize(seed), seed) for seed in range(100))
initialize(seed)
print('seed = {}, initial_loss = {}'.format(seed, loss))

TypeError: model() missing 1 required positional argument: 'dataU'

In [ ]:
# Register hooks to monitor gradient norms.
gradient_norms = defaultdict(list)
for name, value in pyro.get_param_store().named_parameters():
    value.register_hook(lambda g, name=name: gradient_norms[name].append(g.norm().item()))

losses = []
for i in range(200 if not smoke_test else 2):
    loss = svi.step(data,data)
    losses.append(loss)
    print('.' if i % 100 else '\n', end='')

RuntimeError: The size of tensor a (3) must match the size of tensor b (4) at non-singleton dimension 1
Trace Shapes:      
 Param Sites:      
Sample Sites:      
    locs dist   2 |
        value 2 3 |
   scale dist   2 |
        value   2 |
Trace Shapes:
 Param Sites:
Sample Sites:
Trace Shapes:
 Param Sites:
Sample Sites:

In [ ]:
pyplot.figure(figsize=(10,3), dpi=100).set_facecolor('white')
pyplot.plot(losses)
pyplot.xlabel('iters')
pyplot.ylabel('loss')
pyplot.yscale('log')
pyplot.title('Convergence of SVI');

In [ ]:
pyplot.figure(figsize=(10,4), dpi=100).set_facecolor('white')
for name, grad_norms in gradient_norms.items():
    pyplot.plot(grad_norms, label=name)
pyplot.xlabel('iters')
pyplot.ylabel('gradient norm')
pyplot.yscale('log')
pyplot.legend(loc='best')
pyplot.title('Gradient norms during SVI');

In [ ]:
map_estimates = global_guide(data)
weights = map_estimates['weights']
locs = map_estimates['locs']
scale = map_estimates['scale']
print('weights = {}'.format(weights.data.numpy()))
print('locs = {}'.format(locs.data.numpy()))
print('scale = {}'.format(scale.data.numpy()))

In [ ]:
X = np.arange(-3,15,0.1)
Y1 = weights[0].item() * scipy.stats.norm.pdf((X - locs[0].item()) / scale[0].item())
Y2 = weights[1].item() * scipy.stats.norm.pdf((X - locs[1].item()) / scale[1].item())

pyplot.figure(figsize=(10, 4), dpi=100).set_facecolor('white')
pyplot.plot(X, Y1, 'r-')
pyplot.plot(X, Y2, 'b-')
pyplot.plot(X, Y1 + Y2, 'k--')
pyplot.plot(data.data.numpy(), np.zeros(len(data)), 'k*')
pyplot.title('Density of two-component mixture model')
pyplot.ylabel('probability density');